In [1]:
# Import needed libraries
import numpy as np
from scipy.linalg import qr, solve_triangular
from numpy.linalg import svd

In [2]:
# Thruster Class
# Takes in position (default None), toe, cant, and thrust rating and constructs thruster column from cross product of
# force and position after applying toe and cant angle rotations, which is then used to build the matrix A
class Thruster:
    def __init__(self, position=None, toe=0, cant=0, thrust_rating=1.0):
        if position == None:
            default_position = np.array([0.0, 0.0, 0.0])
        else:
            default_position = position

        # If the thruster is pointed towards the other side (-1)
        if default_position[1] < 0:
            toe = -toe

        # Constructs the columns of matrix A by preforming the cross product between the force and position
        force = self.toe_cant_rotations(toe, cant, thrust_rating)
        torque = np.cross(default_position, force)
        self.thrustCol = np.concatenate([force, torque])

    # toe_cant_rotations method finds computes the resulting force from the given rotation angles and thrust rating
    def toe_cant_rotations(self, toe, cant, thrust_rating):
        toe_rad = np.radians(toe)
        cant_rad = np.radians(cant)

        toe_rot_matrix = np.array([
            [np.cos(toe_rad), np.sin(toe_rad), 0.0],
            [-1.0 * np.sin(toe_rad), np.cos(toe_rad), 0.0],
            [0.0, 0.0, 1.0]])

        cant_rot_matrix = np.array([
            [np.cos(cant_rad), 0.0, -1.0 * np.sin(cant_rad)],
            [0.0, 1.0, 0.0],
            [np.sin(cant_rad), 0, np.cos(cant_rad)]])

        rotation_matrix = toe_rot_matrix @ cant_rot_matrix
        return thrust_rating * (rotation_matrix @ np.array([1.0, 0.0, 0.0]))

    def getThruster(self):
        return self.thrustCol



In [3]:
# ROV class takes in thrusters and weights (default [np.ones(6)])
class ROV:
    def __init__(self, thrusters, weights=np.ones(6)):
        self.thrusters = thrusters
        self.weights = weights
        self.thrustMatrix = self.build_thrust_matrix()

    def build_thrust_matrix(self):
        # Builds the entire thrust matrix A through constructing it with the columns created by the thrusters
        columns = []
        for thruster in self.thrusters:
            columns.append(thruster.getThruster())
        columns = np.array(columns)
        # Takes the transpose which forces rows of motion to become columns as specified in project spec
        return columns.T

    def iterate_scaling(self, A_weighted, b_weighted, x_initial):
        # Initializing variables
        x = x_initial.copy()
        # Indices tracks all indices that are remaining, fixed_values tracks the new vals after setting them to 1 or -1
        indices = list(range(len(x)))
        fixed_values = np.zeros(len(x))
        max_iterations = 6 # failsafe
        original_b_weighted = b_weighted.copy()

        for iteration in range(max_iterations):
            # Exits if all components are in bounds
            if np.all(np.abs(x) <= 1):
                break

            # Find the setting most out of range and set to 1.0 or -1.0 depending on its sign
            max_value = 0
            worst_index = -1
            for i in range(len(x)):
                if i in indices and abs(x[i]) > max_value:
                    max_value = abs(x[i])
                    worst_index = i
            if x[worst_index] > 0:
                new_val = 1.0
            else:
                new_val = -1.0
            fixed_values[worst_index] = new_val

            # If there is only 1 value left in indices, exits after setting that value to the correct new_val
            # This way, there isn't a single  out of bounds thrust setting left that is left out from this iteration method
            if len(indices) == 1:
                x[worst_index] = new_val
                break
            # Subtract the projected thrust setting times the corresponding column of the matrix from the right hand side
            # Delete that column from the matrix
            for j in range(len(b_weighted)):
                b_weighted[j] -= A_weighted[j, worst_index] * new_val
            indices.remove(worst_index)
            residual = original_b_weighted - A_weighted @ fixed_values

            # Build the resulting matrix
            A_active = A_weighted[:, indices]

            # Finds rank to decide which type of linear system it is to solve
            m, n = A_active.shape
            u, s, vt = np.linalg.svd(A_active)
            rank = 0
            for val in s:
                if val > s[0] * max(m, n) * 1.0e-12:
                    rank += 1

            if rank < n:
                # Underdetermined uses svd
                u, s, vt = np.linalg.svd(A_active)
                rank_active = 0
                for val in s:
                    if val > s[0] * max(A_active.shape) * 1.0e-12:
                        rank_active += 1
                ranAT = vt[:rank_active].T
                utb = u.T @ residual
                for i in range(rank_active):
                    utb[i] /= s[i]
                x_active = ranAT @ utb[:rank_active]
                # Overdetermined or square uses QR factorization to solve
            else:
                Q, R = np.linalg.qr(A_active)
                qtb = Q.T @ residual
                x_active = np.linalg.solve(R[:n, :n], qtb[:n])

            # updates x after with new values
            x = fixed_values.copy()
            for i in range(len(indices)):
                idx = indices[i]
                x[idx] = x_active[i]
        return x

    # Given a desired movement, attempts to move
    def move(self, desired_move):
        # Initializing components of At = b
        A = self.thrustMatrix
        # vector b is computed by multiplying each of the controller settings with the 1-norm of the corresponding row of A
        row_norms = np.sum(np.abs(A), axis=1)
        b = np.array(desired_move) * row_norms
        m, n = A.shape

        # Scaling
        scaler = np.ones(len(b))
        for i in range(len(scaler)):
            if np.abs(b[i]) > 1.0e-12:
                scaler[i] = 1 / np.sqrt(np.abs(b[i]))

        A_scaled = scaler[:, None] * A
        b_scaled = scaler * b

        sqrt_weights = np.sqrt(self.weights)
        W_sqrt = np.diag(sqrt_weights)
        A_weighted = W_sqrt @ A_scaled
        b_weighted = W_sqrt @ b_scaled

        # Computes rank of A using SVD and given tolerance from numpy documentation
        u, s, vt = np.linalg.svd(A_weighted)
        rank = 0
        for val in s:
            if val > s[0] * max(m, n) * 1.0e-12:
                rank += 1

        # Checking if rank is underdetermined or overdetermined linear system
        # if rank is underdetermined, use SVD to calculate x (approach from homework)
        if rank < n:
            ranAT = vt[:rank].T
            utb = u.T @ b_weighted
            utb[:rank] /= s[:rank]
            xMinimumNorm = ranAT @ utb[:rank]
            kerA = vt[rank:].T
            kernel_dim = A_weighted.shape[1] - rank
            if kernel_dim > 0:
                alpha = np.zeros(kernel_dim)
                x = xMinimumNorm + kerA @ alpha
            else:
                x = xMinimumNorm

        # If rank is overdetermined or square system, solve with QR factorization
        else:
            q, r = qr(A_weighted)
            qtb = q.T @ b_weighted
            n = A.shape[1]
            x = solve_triangular(r[:n, :n], qtb[:n])
        print(x)
        # Making sure that the thruster settings are scaled so they don't exceed their assigned power of -1  to 1
        x = self.iterate_scaling(A_weighted, b_weighted, x)
        # Compute achieved RHS
        rhs_achieved = A @ x

        # Print results of each move operation
        print()
        print("Desired Move = ", desired_move)
        print("Computed thruster settings = ", x)
        print("Desired right hand side = ", b)
        print("Right hand side achieved = ", rhs_achieved)

In [4]:
# Test 1

test1 = ROV([Thruster()])
print("Thrust matrix = ", test1.thrustMatrix)
test1.move([1, 0, 0, 0, 0, 0])
test1.move([0, 1, 0, 0, 0, 0])
test1.move([0, 0, 1, 0, 0, 0])
test1.move([0, 0, 0, 1, 0, 0])
test1.move([0, 0, 0, 0, 1, 0])
test1.move([0, 0, 0, 0, 0, 1])

Thrust matrix =  [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[1.]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [1.]
Desired right hand side =  [1. 0. 0. 0. 0. 0.]
Right hand side achieved =  [1. 0. 0. 0. 0. 0.]
[0.]

Desired Move =  [0, 1, 0, 0, 0, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]
[0.]

Desired Move =  [0, 0, 1, 0, 0, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]
[0.]

Desired Move =  [0, 0, 0, 1, 0, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]
[0.]

Desired Move =  [0, 0, 0, 0, 1, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]
[0.]

Desired Move =  [0, 0, 0, 0, 0, 1]
Computed thruster settings =  [0.]
Desired right h

In [5]:
# Test 2 - A single thruster out on the port side

test2 = ROV([Thruster([0, 1, 0])])
print("Thrust matrix = ", test2.thrustMatrix)
test2.move([1, 0, 0, 0, 0, 0])
test2.move([0, 1, 0, 0, 0, 0])
test2.move([0, 0, 1, 0, 0, 0])
test2.move([0, 0, 0, 1, 0, 0])
test2.move([0, 0, 0, 0, 1, 0])
test2.move([0, 0, 0, 0, 0, 1])
test2.move([1, 0, 0, 0, 0, 1])

Thrust matrix =  [[ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]]
[0.5]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.5]
Desired right hand side =  [1. 0. 0. 0. 0. 0.]
Right hand side achieved =  [ 0.5  0.   0.   0.   0.  -0.5]
[-0.]

Desired Move =  [0, 1, 0, 0, 0, 0]
Computed thruster settings =  [-0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]
[-0.]

Desired Move =  [0, 0, 1, 0, 0, 0]
Computed thruster settings =  [-0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]
[-0.]

Desired Move =  [0, 0, 0, 1, 0, 0]
Computed thruster settings =  [-0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]
[-0.]

Desired Move =  [0, 0, 0, 0, 1, 0]
Computed thruster settings =  [-0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]
[-0.5]

Desired Move =  [0, 0, 0, 0, 0, 1]
Computed thruster se

In [6]:
# Test 3 - Balanced thrusters on both sides

portThruster = Thruster([0.0, 1.0, 0.0])
starboardThruster = Thruster([0.0, -1.0, 0.0])
test3 = ROV([portThruster, starboardThruster])
print("Thrust matrix = ", test3.thrustMatrix)
test3.move([1, 0, 0, 0, 0, 1])
test3.move([1, 1, 0, 0, 0, 0.75])
test3.move([1, 0, 1, 0, 0, 0.5])
test3.move([1, 0, 0, 1, 0, 0.25])
test3.move([1, 0, 0, 0, 1, 0.1])
test3.move([0, 0, 0, 0, 0, -0.8])
test3.move([0.5, 0, 0, 0, 0, 1])

Thrust matrix =  [[ 1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0. -0.]
 [ 0.  0.]
 [-1.  1.]]
[-0.  2.]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [-0.  1.]
Desired right hand side =  [2. 0. 0. 0. 0. 2.]
Right hand side achieved =  [1. 0. 0. 0. 0. 1.]
[0.25 1.75]

Desired Move =  [1, 1, 0, 0, 0, 0.75]
Computed thruster settings =  [0.14285714 1.        ]
Desired right hand side =  [2.  0.  0.  0.  0.  1.5]
Right hand side achieved =  [1.14285714 0.         0.         0.         0.         0.85714286]
[0.5 1.5]

Desired Move =  [1, 0, 1, 0, 0, 0.5]
Computed thruster settings =  [0.33333333 1.        ]
Desired right hand side =  [2. 0. 0. 0. 0. 1.]
Right hand side achieved =  [1.33333333 0.         0.         0.         0.         0.66666667]
[0.75 1.25]

Desired Move =  [1, 0, 0, 1, 0, 0.25]
Computed thruster settings =  [0.6 1. ]
Desired right hand side =  [2.  0.  0.  0.  0.  0.5]
Right hand side achieved =  [1.6 0.  0.  0.  0.  0.4]
[0.9 1.1]

Desired Move =  [1, 0, 0, 0

In [7]:
# Test 4 - An X-wing fighter

test4 = ROV([Thruster([0, 1, 0.5]),
                 Thruster([0, 1, -0.5]),
                 Thruster([0, -1, 0.5]),
                 Thruster([0, -1, -0.5])], [1, 1, 1, 1, 1, 1])
print("Thrust matrix = ", test4.thrustMatrix)
test4.move([1, 0, 0, 0, 0, 1])
test4.move([1, 0, 0, 0, 1, 0])
test4.move([0, 0, 0, 0, 1, 1])
test4.move([1, 0, 0, 0, 1, 1])
test4.move([-0.5, 0, 0, 0, 0.8, 1])
test4.move([0.4, 0, 0, 0, 0, 0])
test4.move([0.4, 0, 0, 0, 0.4, 0.4])

Thrust matrix =  [[ 1.   1.   1.   1. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.  -0.   0. ]
 [ 0.5 -0.5  0.5 -0.5]
 [-1.  -1.   1.   1. ]]
[ 3.04777373e-17 -5.55111512e-17  2.00000000e+00  2.00000000e+00]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [ 2.26623326e-17 -6.79869978e-17  1.00000000e+00  1.00000000e+00]
Desired right hand side =  [4. 0. 0. 0. 0. 4.]
Right hand side achieved =  [2. 0. 0. 0. 0. 2.]
[ 2.00000000e+00 -3.33066907e-16  2.00000000e+00 -1.11022302e-16]

Desired Move =  [1, 0, 0, 0, 1, 0]
Computed thruster settings =  [1.         0.33333333 1.         0.33333333]
Desired right hand side =  [4. 0. 0. 0. 2. 0.]
Right hand side achieved =  [ 2.66666667e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.66666667e-01 -1.11022302e-16]
[ 7.77156117e-16 -2.00000000e+00  2.00000000e+00 -9.99200722e-16]

Desired Move =  [0, 0, 0, 0, 1, 1]
Computed thruster settings =  [-0.33333333 -1.          1.          0.33333333]
Desired right han

In [8]:
# Test 5 - Two balanced thrusters with toe and cant angles
portThruster = Thruster([0.0, 1.0, 0.0], 2.0, 5.0)
starboardThruster = Thruster([0.0, -1.0, 0.0], 2.0, 5.0)
test5 = ROV([portThruster, starboardThruster])
print("Thrust matrix = ", test5.thrustMatrix)
test5.move([1, 0, 0, 0, 0, 1])
test5.move([1, 1, 0, 0, 0, 0.75])
test5.move([1, 0, 1, 0, 0, 0.5])
test5.move([1, 0, 0, 1, 0, 0.25])
test5.move([1, 0, 0, 0, 1, 0.1])
test5.move([0, 0, 0, 0, 0, -0.8])
test5.move([0, 0, 0, 1, 0, 0])
test5.move([0.5, 0, 0, 0, 0, 1])
test5.move([0.5, 0, 0, 0, 0, 0])
test5.move([0.25, 0, 0, 0, 0.25, 0.25])

Thrust matrix =  [[ 0.99558784  0.99558784]
 [-0.03476669  0.03476669]
 [ 0.08715574  0.08715574]
 [ 0.08715574 -0.08715574]
 [ 0.          0.        ]
 [-0.99558784  0.99558784]]
[0.0023501  1.96758946]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [0.0011944 1.       ]
Desired right hand side =  [1.99117569 0.         0.         0.         0.         1.99117569]
Right hand side achieved =  [ 0.99677698  0.03472517  0.08725984 -0.08705164  0.          0.99439871]
[0.23693181 1.73300776]

Desired Move =  [1, 1, 0, 0, 0, 0.75]
Computed thruster settings =  [0.12440216 1.        ]
Desired right hand side =  [1.99117569 0.06953339 0.         0.         0.         1.49338176]
Right hand side achieved =  [ 1.11944112  0.03044164  0.09799811 -0.07631338  0.          0.87173457]
[0.50438317 1.49561683]

Desired Move =  [1, 0, 1, 0, 0, 0.5]
Computed thruster settings =  [0.35592527 1.        ]
Desired right hand side =  [1.99117569 0.         0.17431149 0.         0.       

In [9]:
# Test 6 - Four balanced thrusters, i.e. the "707" configuration

test6 = ROV([Thruster([0, 2, 0]),
                 Thruster([0, 1, 0]),
                 Thruster([0, -1, 0]),
                 Thruster([0, -2, 0])])
print("Thrust matrix = ", test6.thrustMatrix)
test6.move([1, 0, 0, 0, 0, 0])
test6.move([1, 0, 0, 0, 0, 1])
test6.move([0.25, 0, 0, 0, 0, 0])
test6.move([0.57, 0, 0, 0, 0, 0.57])

Thrust matrix =  [[ 1.  1.  1.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -0. -0.]
 [ 0.  0.  0.  0.]
 [-2. -1.  1.  2.]]
[1. 1. 1. 1.]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [1. 1. 1. 1.]
Desired right hand side =  [4. 0. 0. 0. 0. 0.]
Right hand side achieved =  [4. 0. 0. 0. 0. 0.]
[-0.2  0.4  1.6  2.2]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [-1.  1.  1.  1.]
Desired right hand side =  [4. 0. 0. 0. 0. 6.]
Right hand side achieved =  [2. 0. 0. 0. 0. 4.]
[0.25 0.25 0.25 0.25]

Desired Move =  [0.25, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.25 0.25 0.25 0.25]
Desired right hand side =  [1. 0. 0. 0. 0. 0.]
Right hand side achieved =  [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.11022302e-16]
[-0.114  0.228  0.912  1.254]

Desired Move =  [0.57, 0, 0, 0, 0, 0.57]
Computed thruster settings =  [-0.7   0.98  1.    1.  ]
Desired right hand side =  [2.28 0.   0.   0.   0.   3.42]
Righ

In [10]:
# Test 7 - Oh no! The inboard thruster on the starboard side has just failed
 
test7 = ROV([Thruster([0, 2, 0]),
                 Thruster([0, 1, 0]),
                 Thruster([0, -2, 0])])
print("Thrust matrix = ", test7.thrustMatrix)
test7.move([1, 0, 0, 0, 0, 0])
test7.move([1, 0, 0, 0, 0, 1])
test7.move([0.25, 0, 0, 0, 0, 0])
test7.move([0.57, 0, 0, 0, 0, 0.57])

Thrust matrix =  [[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -0.]
 [ 0.  0.  0.]
 [-2. -1.  2.]]
[0.80769231 0.92307692 1.26923077]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.53846154 1.         1.        ]
Desired right hand side =  [3. 0. 0. 0. 0. 0.]
Right hand side achieved =  [ 2.53846154  0.          0.          0.          0.         -0.07692308]
[-0.15384615  0.53846154  2.61538462]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [-1.  1.  1.]
Desired right hand side =  [3. 0. 0. 0. 0. 5.]
Right hand side achieved =  [1. 0. 0. 0. 0. 3.]
[0.20192308 0.23076923 0.31730769]

Desired Move =  [0.25, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.20192308 0.23076923 0.31730769]
Desired right hand side =  [0.75 0.   0.   0.   0.   0.  ]
Right hand side achieved =  [7.50000000e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.11022302e-16]
[-0.08769231  0.30692308  1.49076923]

Desired Move =  [0.57, 0, 0, 0, 0, 0

In [11]:
# Test 8 - The starship Enterprise, sort of

test8 = ROV([Thruster([-1, 1, 1]),
                 Thruster([-1, -1, 1]),
                 Thruster([0, 0, 0])])
print("Thrust matrix = ", test8.thrustMatrix)
test8.move([1, 0, 0, 0, 0, 0])
test8.move([0, 0, 0, 0, 1, 0])
test8.move([0, 0, 0, 0, 0, 1])
test8.move([0.2, 0, 0, 0, 0.5, 0.5])

Thrust matrix =  [[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0. -0.  0.]
 [ 1.  1.  0.]
 [-1.  1.  0.]]
[-1.45362316e-16 -0.00000000e+00  3.00000000e+00]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.25 0.25 1.  ]
Desired right hand side =  [3. 0. 0. 0. 0. 0.]
Right hand side achieved =  [ 1.50000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5.00000000e-01 -1.11022302e-16]
[ 1.  1. -2.]

Desired Move =  [0, 0, 0, 0, 1, 0]
Computed thruster settings =  [ 0.66666667  0.66666667 -1.        ]
Desired right hand side =  [0. 0. 0. 0. 2. 0.]
Right hand side achieved =  [3.33333333e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.33333333e+00 1.11022302e-16]
[-1.00000000e+00  1.00000000e+00  5.55111512e-17]

Desired Move =  [0, 0, 0, 0, 0, 1]
Computed thruster settings =  [-1.00000000e+00  1.00000000e+00  5.55111512e-17]
Desired right hand side =  [0. 0. 0. 0. 0. 2.]
Right hand side achieved =  [-1.66533454e-16  0.00000000e+00  0.00000000e+00  0.000000

In [12]:
# Test 9 - The starship Enterprise again, emphasize forward motion by a factor of 10

test9 = ROV([Thruster([-1, 1, 1]),
                 Thruster([-1, -1, 1]),
                 Thruster([0, 0, 0])],
                 [10, 1, 1, 1, 1, 1])
print("Thrust matrix = ", test9.thrustMatrix)
test9.move([1, 0, 0, 0, 0, 0])
test9.move([0, 0, 0, 0, 1, 0])
test9.move([0, 0, 0, 0, 0, 1])
test9.move([0.2, 0, 0, 0, 0.5, 0.5])

Thrust matrix =  [[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0. -0.  0.]
 [ 1.  1.  0.]
 [-1.  1.  0.]]
[ 3.84592537e-16 -0.00000000e+00  3.00000000e+00]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.76923077 0.76923077 1.        ]
Desired right hand side =  [3. 0. 0. 0. 0. 0.]
Right hand side achieved =  [ 2.53846154e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.53846154e+00 -1.11022302e-16]
[ 1.  1. -2.]

Desired Move =  [0, 0, 0, 0, 1, 0]
Computed thruster settings =  [ 0.52380952  0.52380952 -1.        ]
Desired right hand side =  [0. 0. 0. 0. 2. 0.]
Right hand side achieved =  [0.04761905 0.         0.         0.         1.04761905 0.        ]
[-1.00000000e+00  1.00000000e+00  8.23363432e-17]

Desired Move =  [0, 0, 0, 0, 0, 1]
Computed thruster settings =  [-1.  1.  0.]
Desired right hand side =  [0. 0. 0. 0. 0. 2.]
Right hand side achieved =  [0. 0. 0. 0. 0. 2.]
[-0.   1.  -0.4]

Desired Move =  [0.2, 0, 0, 0, 0.5, 0.5]
Computed thruster sett

In [13]:
# Reading in test files
file_path = "final.txt"
with open(file_path, 'r') as file:
    lines = file.readlines()

num_of_thrusters = int(lines[0])
thrusters = []
for i in range(num_of_thrusters):
    settings = lines[i + 1].split()
    x = int(settings[0])
    y = int(settings[1])
    z = int(settings[2])
    toe = int( settings[3])
    cant = float(settings[4])
    rating = float(settings[5])
    thrusters.append(Thruster([x, y, z], toe, cant, rating))
weights = [int(weight) for weight in lines[num_of_thrusters + 1].split()]
num_moves = int(lines[num_of_thrusters + 2])
range_start = num_of_thrusters + 3
test = ROV(thrusters, weights)
print("Thrust matrix = ", test.thrustMatrix)
for i in range(range_start, len(lines)):
    move = [float(move) for move in lines[i].split()]
    test.move(move)

Thrust matrix =  [[ 9.84657762e-01  9.99390827e-01  9.99390827e-01  9.84657762e-01
   6.00000000e-01  4.00000000e-01  1.53080850e-17  1.53080850e-17]
 [-1.71872652e-02 -3.48994967e-02  3.48994967e-02  1.71872652e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.73648178e-01  0.00000000e+00  0.00000000e+00  1.73648178e-01
   0.00000000e+00  0.00000000e+00  2.50000000e-01 -2.50000000e-01]
 [ 3.47296355e-01  0.00000000e+00 -0.00000000e+00 -3.47296355e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   6.00000000e-01 -4.00000000e-01 -2.50000000e-01 -2.50000000e-01]
 [-1.96931552e+00 -9.99390827e-01  9.99390827e-01  1.96931552e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00]]
[ 0.87678202  1.31924174  1.31924174  0.87678202  0.5464376   0.69174497
 -0.50667726  0.71133554]

Desired Move =  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Computed thruster settings =  [ 1.   